<a href="https://colab.research.google.com/github/ivansirait/Analisis_kebiasaan_saat_belajar_vs_Prestasi_Akademik-Siswa/blob/Try-before-Main/DasiDalis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
import pandas as pd
from google.colab import files

# uploaded = files.upload()

# Ambil nama file
# filename = list(uploaded.keys())[0]

# Membaca file Excel
# df = pd.read_csv(filename)
df = pd.read_csv("tes.csv")

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1252 entries, 0 to 1251
Data columns (total 22 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   Timestamp                                                      1252 non-null   object 
 1   Nama Lengkap                                                   1249 non-null   object 
 2   Jenis Kelamin                                                  1252 non-null   object 
 3   Jenis Sekolah Menengah atas yang anda tempuh                   1251 non-null   object 
 4   Asal Sekolah                                                   1248 non-null   object 
 5   Tingkat Kelas                                                  1249 non-null   object 
 6   Gaya belajar yang biasanya digunakan                           1247 non-null   object 
 7   Teknik belajar yang biasanya digunakan                      

In [111]:
print("Missing values awal:")
print(df.isnull().sum())

print("Jumlah nama duplikat:", df["Nama Lengkap"].duplicated().sum())

df = df.drop_duplicates(subset="Nama Lengkap", keep="first")

print("Duplikat setelah dibersihkan:", df["Nama Lengkap"].duplicated().sum())
df.info()


Missing values awal:
Timestamp                                                          0
Nama Lengkap                                                       3
Jenis Kelamin                                                      0
Jenis Sekolah Menengah atas yang anda tempuh                       1
Asal Sekolah                                                       4
Tingkat Kelas                                                      3
Gaya belajar yang biasanya digunakan                               5
Teknik belajar yang biasanya digunakan                             5
Apakah kamu belajar sendiri atau dengan bantuan orang lain?        1
Lokasi Belajar                                                     4
Jam Belajar                                                        2
Total Waktu untuk mengerjakan Pekerjaan Rumah(PR)/Hari             1
Total Waktu untuk Belajar Mandiri /Hari                            2
Catatan singkat tentang Belajar                                  247
Mood Saat Bel

In [112]:
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].astype(str).str.strip()

In [114]:
numeric_cols = [
    "Mood Saat Belajar",
    "Pikiran atau Stres Akademik",
    "Jumlah prestasi di dalam sekolah *",
    "Jumlah prestasi di luar sekolah"
]

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

if "Mood Saat Belajar" in df.columns:
    df["Mood Saat Belajar"] = df["Mood Saat Belajar"].fillna(3).astype(int)
if "Pikiran atau Stres Akademik" in df.columns:
    df["Pikiran atau Stres Akademik"] = df["Pikiran atau Stres Akademik"].fillna(3).astype(int)

if "Jumlah prestasi di dalam sekolah *" in df.columns:
    df["Jumlah prestasi di dalam sekolah *"] = df["Jumlah prestasi di dalam sekolah *"].fillna(0).astype(int)
if "Jumlah prestasi di luar sekolah" in df.columns:
    df["Jumlah prestasi di luar sekolah"] = df["Jumlah prestasi di luar sekolah"].fillna(0).astype(int)

In [115]:
df["Rata rata nilai Siswa(nilai tarakhir)"] = (
    df["Rata rata nilai Siswa(nilai tarakhir)"]
    .astype(str)
    .str.replace(",", ".", regex=False)
    .pipe(pd.to_numeric, errors="coerce")
)

df = df.dropna(subset=["Rata rata nilai Siswa(nilai tarakhir)"])


In [116]:
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    return df[
        (df[column] >= lower_bound) &
        (df[column] <= upper_bound)
    ]

target_col = "Rata rata nilai Siswa(nilai tarakhir)"
print("Jumlah data sebelum outlier:", df.shape[0])
df = remove_outliers_iqr(df, target_col)
print("Jumlah data setelah outlier:", df.shape[0])


Jumlah data sebelum outlier: 915
Jumlah data setelah outlier: 889


In [117]:
def normalize_text(x):
    if pd.isna(x):
        return np.nan
    x = unicodedata.normalize("NFKC", str(x).lower().strip())
    return x if x != "nan" else np.nan

for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].apply(normalize_text)


In [118]:
def map_gaya(x):
    if pd.isna(x): return np.nan
    if "visual" in x: return "visual"
    if "auditory" in x: return "auditory"
    if "kinestetik" in x: return "kinestetik"
    return "lainnya"

df["Gaya belajar yang biasanya digunakan"] = df["Gaya belajar yang biasanya digunakan"].apply(map_gaya)


def map_teknik(x):
    if pd.isna(x): return np.nan
    if "pomodoro" in x or "podomoro" in x: return "podomoro"
    if "active recall" in x: return "active_recall"
    if "feynman" in x: return "feynman"
    if "latihan soal" in x: return "latihan_soal"
    if "spaced" in x: return "spaced_repetition"
    return "lainnya"

df["Teknik belajar yang biasanya digunakan"] = df["Teknik belajar yang biasanya digunakan"].apply(map_teknik)


In [119]:
def map_lokasi(x):
    if pd.isna(x): return "lainnya"
    if any(k in x for k in ["rumah", "kamar", "kost", "meja"]): return "rumah"
    if any(k in x for k in ["sekolah", "kelas", "bengkel"]): return "sekolah"
    if "asrama" in x: return "asrama"
    if any(k in x for k in ["les", "bimbel"]): return "bimbel_les"
    if "perpustakaan" in x: return "perpustakaan"
    return "lainnya"

df["Lokasi Belajar"] = df["Lokasi Belajar"].apply(map_lokasi)


def map_asal_sekolah(x):
    if pd.isna(x): return "lainnya"
    if "bintang timur" in x and "sma" in x: return "sma_sw_bintang_timur_balige"
    if "yapim" in x and "smk" in x: return "smk_yapim_balige"
    if "laguboti" in x and "smk" in x and "1" in x: return "smk_n_1_laguboti"
    if "arjuna" in x and "smk" in x: return "smk_arjuna_laguboti"
    return "lainnya"

df["Asal Sekolah"] = df["Asal Sekolah"].apply(map_asal_sekolah)


In [120]:
mapping_waktu = {
    "< 15 menit": 1,
    "15 - 30 menit": 2,
    "30 - 60 menit": 3,
    "> 60 menit": 4
}

for col in [
    "Total Waktu untuk mengerjakan Pekerjaan Rumah(PR)/Hari",
    "Total Waktu untuk Belajar Mandiri /Hari"
]:
    df[col] = df[col].map(mapping_waktu)

df["total_waktu_belajar"] = (
    df["Total Waktu untuk mengerjakan Pekerjaan Rumah(PR)/Hari"] +
    df["Total Waktu untuk Belajar Mandiri /Hari"]
)


In [123]:
one_hot_cols = [
    "Gaya belajar yang biasanya digunakan",
    "Teknik belajar yang biasanya digunakan",
    "Jenis Kelamin",
    "Jam Belajar",
    "Jenis Sekolah Menengah atas yang anda tempuh",
    "Apakah kamu belajar sendiri atau dengan bantuan orang lain? ",
    "Lokasi Belajar",
    "Asal Sekolah",
    "Apakah Anda pernah meraih prestasi di dalam atau luar sekolah"
]

df = pd.get_dummies(df, columns=one_hot_cols, drop_first=False)


In [124]:
active_learning_cols = [
    c for c in df.columns
    if c.startswith("Teknik belajar yang biasanya digunakan_")
    and c.split("_")[-1] in [
        "feynman", "latihan_soal",
        "spaced_repetition", "active_recall", "podomoro"
    ]
]

df["active_learning_score"] = df[active_learning_cols].sum(axis=1)

df[["total_waktu_belajar", "active_learning_score"]].head()


,total_waktu_belajar,active_learning_score
0,5,0
1,4,0
2,3,0
3,3,0
5,7,0


In [125]:
from sklearn.preprocessing import MinMaxScaler

num_cols = df.select_dtypes(include=["int64", "float64"]).columns

scaler = MinMaxScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 889 entries, 0 to 1251
Data columns (total 48 columns):
 #   Column                                                                            Non-Null Count  Dtype  
---  ------                                                                            --------------  -----  
 0   Timestamp                                                                         889 non-null    object 
 1   Nama Lengkap                                                                      888 non-null    object 
 2   Tingkat Kelas                                                                     888 non-null    object 
 3   Total Waktu untuk mengerjakan Pekerjaan Rumah(PR)/Hari                            889 non-null    float64
 4   Total Waktu untuk Belajar Mandiri /Hari                                           889 non-null    float64
 5   Catatan singkat tentang Belajar                                                   752 non-null    object 
 6   Mood S

In [ ]:
print(df.columns)


In [ ]:

# 2. Rename Kolom (Opsional tapi disarankan untuk mempermudah coding)
# Kita akan menyederhanakan nama kolom utama yang akan dianalisis
df.rename(columns={
    'Jenis Sekolah Menengah atas yang anda tempuh': 'Jenis_Sekolah',
    'Gaya belajar yang biasanya digunakan': 'Gaya_Belajar',
    'Rata rata nilai Siswa(nilai tarakhir)': 'Nilai_Rata_Rata',
    'Total Waktu untuk Belajar Mandiri /Hari': 'Waktu_Belajar_Mandiri',
    'Pikiran atau Stres Akademik': 'Stres_Akademik',
    'Jenis Kelamin': 'Jenis_Kelamin'
}, inplace=True)

print("\n--- Ringkasan Statistik Nilai Rata-Rata ---")
print(df['Nilai rata-rata (Terakhir)'].describe())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# 2. Rename Kolom untuk Analisis dan Plotting (mempermudah penulisan kode)
df.rename(columns={
    'Rata - rata nilai siswa': 'Nilai_Rata_Rata',
    'Jam Belajar Siswa per Hari': 'Waktu_Belajar_Harian',
    'Gaya belajar yang biasanya digunakan': 'Gaya_Belajar',
    'Stress Akademik': 'Stres_Akademik',
    'Tingkat Kelas': 'Tingkat_Kelas',
    'Jenis Sekolah Menengah atas yang anda tempuh': 'Jenis_Sekolah',
    'Pernah meraih prestasi?': 'Pernah_Prestasi'
}, inplace=True)

print("✅ Data Bersih dan Nama Kolom Siap untuk Analisis.")

# --- Visualisasi Utama untuk Insight ---
# B. Perbandingan Nilai Rata-Rata Berdasarkan Gaya Belajar
plt.figure(figsize=(12, 7))
sns.boxplot(x='Gaya_Belajar', y='Nilai rata-rata (Terakhir)', data=df, palette='viridis')
plt.title('2. Nilai Rata-Rata Berdasarkan Gaya Belajar', fontsize=15)
plt.xlabel('Gaya Belajar', fontsize=12)
plt.ylabel('Nilai Rata-Rata', fontsize=12)
plt.show()

avg_nilai_gaya = df.groupby('Gaya_Belajar')['Nilai rata-rata (Terakhir)'].mean().sort_values(ascending=False)
print("\n--- Rata-Rata Nilai per Gaya Belajar ---")
print(avg_nilai_gaya)
print("💡 Insight B: Menentukan gaya belajar mana yang berkorelasi dengan nilai tertinggi (fokus pada median dan outliers).")


In [ ]:
# C. Dampak Stres Akademik terhadap Nilai Rata-Rata
plt.figure(figsize=(8, 6))
sns.barplot(x='Stres_Akademik', y='Nilai rata-rata (Terakhir)', data=df, palette='Blues')
plt.title('3. Dampak Stres Akademik terhadap Nilai Rata-Rata', fontsize=15)
plt.xlabel('Stres Akademik', fontsize=12)
plt.ylabel('Nilai rata-rata (Terakhir)', fontsize=12)
plt.show()

avg_nilai_stres = df.groupby('Stres_Akademik')['Nilai rata-rata (Terakhir)'].mean().sort_values(ascending=False)
print("\n--- Rata-Rata Nilai per Tingkat Stres ---")
print(avg_nilai_stres)
print("💡 Insight C: Melihat apakah tingkat stres (Tinggi/Sedang/Rendah) memengaruhi performa secara signifikan.")



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Mengatur ukuran grafik
plt.figure(figsize=(8, 6))

# Membuat boxplot untuk menunjukkan distribusi nilai rata-rata berdasarkan tingkat Stres Akademik
sns.boxplot(x='Stres_Akademik', y='Nilai rata-rata (Terakhir)', data=df, palette='Blues')

# Menambahkan judul dan label sumbu
plt.title('3. Dampak Stres Akademik terhadap Nilai Rata-Rata', fontsize=15)
plt.xlabel('Stres Akademik', fontsize=12)
plt.ylabel('Nilai rata-rata (Terakhir)', fontsize=12)

# Menampilkan grafik
plt.show()


In [ ]:
# Mengatur ukuran grafik
plt.figure(figsize=(8, 6))

# Membuat boxplot
sns.boxplot(x='Mood Saat Belajar', y='Rata rata nilai Siswa(nilai tarakhir)', data=df, palette='coolwarm')

# Menambahkan judul dan label
plt.title('Pengaruh Mood terhadap Nilai Akademik', fontsize=15)
plt.xlabel('Mood saat belajar', fontsize=12)
plt.ylabel('Nilai rata-rata (Terakhir)', fontsize=12)

# Menampilkan grafik
plt.show()


In [ ]:
# D. Korelasi Waktu Belajar Harian vs. Nilai Rata-Rata
plt.figure(figsize=(8, 6))
sns.scatterplot(x='Waktu_Belajar_Harian', y='Rata rata nilai Siswa(nilai tarakhir)', data=df, alpha=0.6, color='darkorange')
plt.title('4. Korelasi Waktu Belajar Harian vs. Nilai Rata-Rata', fontsize=15)
plt.xlabel('Waktu Belajar Harian (Jam)', fontsize=12)
plt.ylabel('Nilai rata-rata (Terakhir)', fontsize=12)
plt.show()

korelasi = df['Waktu_Belajar_Harian'].corr(df['Nilai rata-rata (Terakhir)'])
print(f"\n--- Koefisien Korelasi Waktu Belajar Harian vs. Nilai: {korelasi:.3f} ---")
print("💡 Insight D: Mengukur kekuatan hubungan antara kuantitas waktu belajar dengan hasil akademik.")